# wps_convolution

#### wps_convolution is a process that runs the [convolution](https://rvic.readthedocs.io/en/latest/user-guide/convolution/) module of a VIC streamflow routing model, called [RVIC](https://rvic.readthedocs.io/en/latest/). To get started, first instatiate the client. Here, the client will try to connect to a remote osprey instance using the url parameter.

In [1]:
from birdy import WPSClient
import requests
import os
from datetime import datetime
from wps_tools.utils import copy_http_content
from netCDF4 import Dataset
from tempfile import NamedTemporaryFile
from rvic.core.config import read_config
from pkg_resources import resource_filename

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "osprey":
    os.chdir('../')

url = 'http://localhost:5002/wps'
osprey = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [2]:
# NBVAL_IGNORE_OUTPUT
osprey.convolution?

Signature: osprey.convolution(loglevel='INFO', config_file=None, config_dict=None)
Docstring:
Aggregates the flow contribution from all upstream grid cellsat every timestep lagged according the Impuls Response Functions.

Parameters
----------
config_file : ComplexData:mimetype:`text/x-cfg`
    Path to input configuration file for Convolution process
config_dict : string
    Dictionary containing input configuration for Convolution process
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    Output Netcdf File
File:      ~/Desktop/osprey/</home/sangwonl/Desktop/osprey/venv/lib/python3.7/site-packages/birdy/client/base.py-1>
Type:      method


#### We can use the docstring to ensure we provide the appropriate parameters.

In [ ]:
# run convolution with local configuration file
output_config_file = osprey.convolution(config_file = resource_filename("tests", "/data/configs/convolve_opendap.cfg"))

In [ ]:
# run convolution with local configuration dictionary
config_dict = {
    "OPTIONS": {
        "CASEID": "sample",
        "CALENDAR": "standard",
        "RUN_STARTDATE": "2012-12-01-00",
        "STOP_DATE": "2012-12-31",
    },
    "DOMAIN": {
        "FILE_NAME": "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/RVIC/sample_routing_domain.nc",
    },
    "PARAM_FILE":{
        "FILE_NAME": "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/RVIC/sample.rvic.prm.COLUMBIA.20180516.nc",
    },
    "INPUT_FORCINGS":{
        "DATL_PATH": "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/RVIC/",
        "DATL_FILE": "columbia_vicset2.nc",
    },
}
output_config_dict = osprey.convolution(config_dict = config_dict)

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [ ]:
input_config = read_config("tests/data/configs/convolve_opendap.cfg")

# Expected output metadata determined from the input configuration file
expected = {
    "title": "RVIC history file",
    "year": datetime.now().strftime("%Y"),
    "month": datetime.now().strftime("%b"),
    "day": datetime.now().strftime("%d").zfill(2),
    "RvicDomainFile": input_config["DOMAIN"]["FILE_NAME"].split("/")[-1],
}

In [ ]:
def test_convolution_output(output, expected):
    with NamedTemporaryFile(suffix=".nc", prefix="tmp_copy", dir="/tmp", delete=True) as tmp_file:
        output_data = Dataset(copy_http_content(output.get()[0], tmp_file))

        # Metadata of the output of RVIC convolution module
        metadata = {
            "title": output_data.title,
            "year": output_data.history.split()[5],
            "month": output_data.history.split()[2].zfill(2),
            "day": output_data.history.split()[3].zfill(2),
            "RvicDomainFile": output_data.RvicDomainFile,
        }
        assert metadata == expected

In [ ]:
test_convolution_output(output_config_file, expected)
test_convolution_output(output_config_dict, expected)